In [ ]:
!sudo apt install pciutils lshw
!curl -fsSL https://ollama.com/install.sh | sh
!pip install -U langchain-ollama

In [ ]:
pip install -U langchain-community

In [3]:
import threading
import subprocess
import time

def run_ollama_serve():
    subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [4]:
!ollama --version

ollama version is 0.6.8


In [5]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 970aa74c0a90: 100% ▕▏ 274 MB                         
pulling c71d239df917: 100% ▕▏  11 KB                         
pulling ce4a164fc046: 100% ▕▏   17 B                         
pulling 31df23ea7daa: 100% ▕▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
!ollama pull llama3.1:8b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 6a0746a1ec1a: 100% ▕▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕▏  12 KB                         
pulling 8ab4849b038c: 100% ▕▏  254 B                         
pulling 577073ffcc6c: 100% ▕▏  110 B                         
pulling 3f8eb4da87fa: 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [7]:
pip install faiss-cpu

In [ ]:
pip install langchain_experimental

In [ ]:
!pip install --upgrade langchain

In [ ]:
pip install google-search-results

In [ ]:
from langchain.document_loaders import TextLoader
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, AgentType
from langchain_experimental.tools import PythonREPLTool
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain_experimental.tools import PythonREPLTool
import os

In [ ]:
os.environ["SERPAPI_API_KEY"] = "SERPAPIKEY"

In [13]:
# --- Step 1: Load text documents ---
file = 'merged_langchain_docs.txt'
documents = []

loader = TextLoader(file)
documents.extend(loader.load())

In [14]:
# --- Step 2: Chunk documents for better embedding ---
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(documents)

In [15]:
# --- Step 3: Create embeddings and FAISS vector DB ---
embedding_model = OllamaEmbeddings(model="nomic-embed-text")
vectordb = FAISS.from_documents(split_docs, embedding_model)
vectordb.save_local("faiss_index")

In [16]:
# --- Step 4: Set up tools ---
retriever_tool = vectordb.as_retriever(search_type="similarity", k=3)

langchain_doc_tool = Tool(
    name="LangChainDocs",
    func=retriever_tool.get_relevant_documents,
    description="Useful for answering any question about LangChain, e.g, its features, usage, components, and examples."
)

search = SerpAPIWrapper()
search_tool = Tool(
    name="Search",
    func=search.run,
    description="Useful for answering questions (not about LangChain) about current events or general knowledge from the web."
)

python_tool = PythonREPLTool()

In [ ]:
# --- Step 5: Load LLM ---
llm = ChatOllama(model="llama3.1:8b", temperature=0)

In [18]:
# --- Step 6: Memory for chat history ---
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

<ipython-input-18-916320d495e8>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [19]:
# --- Step 7: Initialize agent with tools ---
tools = [search_tool, python_tool, langchain_doc_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

<ipython-input-19-a6dc22fdfe94>:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [20]:
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    response = agent.run(query)
    print("Bot:", response)

You: what are langchain features


<ipython-input-20-17268432387c>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)




> Entering new AgentExecutor chain...
```json
{
    "action": "LangChainDocs",
    "action_input": ""
}
```

(Note: I've responded in Option 1 format, as the user's input is about LangChain, so it's best to use the `LangChainDocs` tool for this question.)
Observation: [Document(id='72503380-5ff5-47bc-8f12-a9f5ebe56b34', metadata={'source': 'merged_langchain_docs.txt'}, page_content='}'), Document(id='49a56077-de3a-46d4-a719-d6d6fb964389', metadata={'source': 'merged_langchain_docs.txt'}, page_content='----'), Document(id='7f5fe472-ce67-4d44-9941-e7d0fdd6482b', metadata={'source': 'merged_langchain_docs.txt'}, page_content='----'), Document(id='4a7570c8-04c1-444a-ab9c-4f5eef3aaee5', metadata={'source': 'merged_langchain_docs.txt'}, page_content='Explanation:\n\u200b')]
Thought:```json
{
    "action": "Final Answer",
    "action_input": "LangChain features include its ability to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversat

```json
{
    "action": "Python_REPL",
    "action_input": "print([i for i,x in enumerate([22,3,4,5,7]) if x == 3][0])"
}
```

This will execute the Python command and print the index of number 3 in the list.
Observation: 1

Thought:```json
{
    "action": "Final Answer",
    "action_input": "The index is 1."
}
```

> Finished chain.
Bot: The index is 1.
You: exit


In [ ]:
pip install gradio

In [ ]:
import gradio as gr

def chat_fn(message, history):
    try:
        response = agent.run(message)
    except Exception as e:
        response = f"[ERROR] {str(e)}"
    return response

iface = gr.ChatInterface(
    fn=chat_fn,
    title="LangChain RAG Chatbot",
)
iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a76c266739a22da7ec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# iface.close()

Closing server running on port: 7860


In [28]:
print(memory.buffer)

[HumanMessage(content='what are langchain features', additional_kwargs={}, response_metadata={}), AIMessage(content='LangChain features include its ability to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.', additional_kwargs={}, response_metadata={}), HumanMessage(content='who is donald trump', additional_kwargs={}, response_metadata={}), AIMessage(content="Donald Trump is an American businessman and politician who served as the 45th President of the United States from 2017 to 2021. He was born on June 14, 1946, in Queens, New York. Trump's presidency was marked by controversy and division, with many criticizing his policies and behavior.", additional_kwargs={}, response_metadata={}), HumanMessage(content='how old is he', additional_kwargs={}, response_metadata={}), AIMessage(content='Donald Trump was born on June 14, 1946', additional_kwargs={},